# 线性回归——使用Gluon

[前一章](linear-regression-scratch.md)我们仅仅使用了`ndarray`和`autograd`来实现线性回归，这一章我们仍然实现同样的模型，但是使用高层抽象包`gluon`。

## 创建数据集

我们生成同样的数据集

In [13]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

data_root = '../data'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
for i in range(150):
    txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % i
    with open(txt_filename1, 'r') as f:
        txts = f.read().split()

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape=y.shape)


[[[-0.41213271  1.52278841]
  [ 1.24897444 -0.33277884]
  [-0.48362762 -0.92140299]]

 [[ 0.25383762 -0.03598363]
  [-0.11660124 -0.2931439 ]
  [ 1.48245633 -0.58981359]]

 [[ 0.81097436 -0.52307099]
  [ 1.12914741  0.03926884]
  [ 0.16556863 -0.97811055]]

 ..., 
 [[-0.11661945  2.1543231 ]
  [-1.02014947  1.11945248]
  [-1.32146716  0.7630707 ]]

 [[ 1.38129365  2.0401516 ]
  [ 0.47268888 -0.54245782]
  [-0.37366498  0.15885045]]

 [[-1.82788789  1.38307118]
  [-0.65179902 -0.29327434]
  [-0.38820505 -0.48204207]]]
<NDArray 1000x3x2 @cpu(0)>

[[ -0.8619889    8.38162613]
 [  5.0962925    5.11252403]
 [  1.98307872   3.02418923]
 ..., 
 [  7.42876911   4.70107698]
 [  5.33990431  10.12491703]
 [  2.77331471   7.97042847]]
<NDArray 1000x2 @cpu(0)>


## 数据读取

但这里使用`data`模块来读取数据。

In [2]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

读取跟前面一致：

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[ 1.0630331   0.17892691]
 [-1.86081767  1.38656259]
 [ 2.58488894 -1.14167035]
 [ 0.5074473   0.07947154]
 [-1.7132839   0.08990741]
 [-1.07668817 -0.75819719]
 [-0.6686148   1.38497305]
 [-0.29596215  1.83803487]
 [-1.35992277  0.57972157]
 [ 0.09210896 -0.1585041 ]]
<NDArray 10x2 @cpu(0)> 
[  5.72280836  -4.23733759  13.25807858   4.94102526   0.46718451
   4.63486099  -1.83509505  -2.64578891  -0.48840913   4.92344046]
<NDArray 10 @cpu(0)>


## 定义模型

之前一章中，当我们从0开始训练模型时，需要先声明模型参数，然后再使用它们来构建模型。但`gluon`提供大量预定义的层，我们只需要关注使用哪些层来构建模型。例如线性模型就是使用对应的`Dense`层；之所以称为dense层，是因为输入的所有节点都与后续的节点相连。在这个例子中仅有一个输出，但在大多数后续章节中，我们会用到具有多个输出的网络。

我们之后还会介绍如何构造任意结构的神经网络，但对于初学者来说，构建模型最简单的办法是利用`Sequential`来所有层串起来。输入数据之后，`Sequential`会依次执行每一层，并将前一层的输出，作为输入提供给后面的层。首先我们定义一个空的模型：

In [4]:
net = gluon.nn.Sequential()

然后我们加入一个`Dense`层，它唯一必须定义的参数就是输出节点的个数，在线性模型里面是1.

In [5]:
net.add(gluon.nn.Dense(1))

（注意这里我们并没有定义说这个层的输入节点是多少，这个在之后真正给数据的时候系统会自动赋值。我们之后会详细介绍这个特性是如何工作的。）

## 初始化模型参数

在使用前`net`我们必须要初始化模型权重，这里我们使用默认随机初始化方法（之后我们会介绍更多的初始化方法）。

In [6]:
net.initialize()

## 损失函数

`gluon`提供了平方误差函数：

In [7]:
square_loss = gluon.loss.L2Loss()

## 优化

同样我们无需手动实现随机梯度下降，我们可以创建一个`Trainer`的实例，并且将模型参数传递给它就行。

In [8]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练
使用`gluon`使模型训练过程更为简洁。我们不需要挨个定义相关参数、损失函数，也不需使用随机梯度下降。`gluon`的抽象和便利的优势将随着我们着手处理更多复杂模型的愈发显现。不过在完成初始设置后，训练过程本身和前面没有太多区别，唯一的不同在于我们不再是调用`SGD`，而是`trainer.step`来更新模型（此处一并省略之前绘制损失变化的折线图和散点图的过程，有兴趣的同学可以自行尝试）。

In [9]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 0.909584
Epoch 1, average loss: 0.000051
Epoch 2, average loss: 0.000051
Epoch 3, average loss: 0.000051
Epoch 4, average loss: 0.000051


比较学到的和真实模型。我们先从`net`拿到需要的层，然后访问其权重和位移。

In [10]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 2.00104523 -3.3992312 ]]
 <NDArray 1x2 @cpu(0)>)

In [11]:
true_b, dense.bias.data()

(4.2, 
 [ 4.20073891]
 <NDArray 1 @cpu(0)>)

## 结论

可以看到`gluon`可以帮助我们更快更干净地实现模型。


## 练习

- 在训练的时候，为什么我们用了比前面要大10倍的学习率呢？（提示：可以尝试运行 `help(trainer.step)`来寻找答案。）
- 如何拿到`weight`的梯度呢？（提示：尝试 `help(dense.weight)`）

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/742)